In [7]:
# Code For Published Data Timeline

# Required Libraries
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
warnings.filterwarnings('ignore')

# !!!!Download Location!!!!
download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")
# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")
# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')


first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)


second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


In [ ]:
/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]
/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]
/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[2]/div[2]


/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[10]/div[2]
,/html/body/div[1]/div/div[4]/div/div[2]/button[2]

In [16]:
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np

warnings.filterwarnings('ignore')

# !!!!Download Location!!!!
download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')


# First click
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)

# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)



# BeautifulSoup ile verileri çekme
soup = BeautifulSoup(browser.page_source, 'html.parser')

# class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

# class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

# İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
max_length = max(len(class_1_data), len(class_2_data))
class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

# Verileri bir DataFrame'e koyma
df = pd.DataFrame({
    'Class_1_Data': class_1_data,
    'Class_2_Data': class_2_data
})

# DataFrame'i yazdırma
print(df)

# Zero click
time.sleep(30)
zero_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[4]/div/div[2]/button[2]"))
)
browser.execute_script("arguments[0].click();", zero_click)

# Third click
third_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", third_click)


# Tarayıcıyı kapatma
browser.quit()


                                         Class_1_Data Class_2_Data
0                                   Textiles (0 HS92)     (0 HS92)
1                                      Silk (50 HS92)    (50 HS92)
2                                      Wool (51 HS92)    (51 HS92)
3                                    Cotton (52 HS92)    (52 HS92)
4            Other vegetable textile fibres (53 HS92)    (53 HS92)
5                        Man-made filaments (54 HS92)    (54 HS92)
6                    Man-made staple fibres (55 HS92)    (55 HS92)
7               Wadding, felt and nonwovens (56 HS92)    (56 HS92)
8                                   Carpets (57 HS92)    (57 HS92)
9                     Special woven fabrics (58 HS92)    (58 HS92)
10  Impregnated, coated or laminated textile fabri...    (59 HS92)
11                          Knitted fabrics (60 HS92)    (60 HS92)
12                            Apparel, knit (61 HS92)    (61 HS92)
13                        Apparel, not knit (62 HS92)    (62 H

In [17]:
# Çöp

import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup
import numpy as np

warnings.filterwarnings('ignore')

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")

# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")

# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://atlas.cid.harvard.edu/explore?country=104&queryLevel=location&product=undefined&year=2019&productClass=HS&target=Product&partner=undefined&startYear=undefined')

# First click
first_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/div[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", first_click)

# Second click
second_click = WebDriverWait(browser, 30).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[1]/div[2]"))
)
browser.execute_script("arguments[0].click();", second_click)

# BeautifulSoup ile ilk veriyi çekme
def get_data(soup):
    # class="_1_ZzYyrQdcPl2PdBK2gqe9" olan veriler
    class_1_data = [element.get_text() for element in soup.find_all(class_="_1_ZzYyrQdcPl2PdBK2gqe9")]

    # class="_1od7BAQOD-GTUQz0_QqSSJ" olan veriler
    class_2_data = [element.get_text() for element in soup.find_all(class_="_1od7BAQOD-GTUQz0_QqSSJ")]

    # İki listeyi eşitleme (eksik elemanları NaN ile dolduruyoruz)
    max_length = max(len(class_1_data), len(class_2_data))
    class_1_data.extend([np.nan] * (max_length - len(class_1_data)))
    class_2_data.extend([np.nan] * (max_length - len(class_2_data)))

    return pd.DataFrame({
        'Class_1_Data': class_1_data,
        'Class_2_Data': class_2_data
    })

# Veri toplama döngüsü
all_data = pd.DataFrame()  # Tüm verilerin toplandığı DataFrame

# 1'den 10'a kadar döngü ile veriyi çekiyoruz
for i in range(1, 11):  # li[1] ile li[10] arasındaki verileri çeker
    # XPath oluşturma
    xpath = f"/html/body/div[1]/div/div[1]/div[4]/div/div/div[3]/div/div/ul/div/div[1]/li[{i}]/div[2]"
    
    # Her öğeyi tıkla
    try:
        item_click = WebDriverWait(browser, 30).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        browser.execute_script("arguments[0].click();", item_click)
        
        # Verileri çekmek için BeautifulSoup ile sayfa kaynağını al
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        df = get_data(soup)
        
        # Yeni çekilen veriyi all_data DataFrame'ine ekle
        all_data = pd.concat([all_data, df], ignore_index=True)
        
        # Öğeyi kapat (3. tıklama, kapatma işlemi)
        browser.execute_script("arguments[0].click();", item_click)
    
    except TimeoutException:
        print(f"Element {i} bulunamadı, devam ediliyor.")
        continue

# Tüm verileri yazdır
print(all_data)

# Tarayıcıyı kapatma
browser.quit()


                                 Class_1_Data Class_2_Data
0                           Textiles (0 HS92)     (0 HS92)
1                              Silk (50 HS92)    (50 HS92)
2                              Wool (51 HS92)    (51 HS92)
3                            Cotton (52 HS92)    (52 HS92)
4    Other vegetable textile fibres (53 HS92)    (53 HS92)
..                                        ...          ...
373                         Vehicles (6 HS92)     (6 HS92)
374                        Machinery (7 HS92)     (7 HS92)
375                      Electronics (8 HS92)     (8 HS92)
376                            Other (9 HS92)     (9 HS92)
377                                  Services          NaN

[378 rows x 2 columns]


In [20]:
all_data.head(50)

,Class_1_Data,Class_2_Data
0,Textiles (0 HS92),(0 HS92)
1,Silk (50 HS92),(50 HS92)
2,Wool (51 HS92),(51 HS92)
3,Cotton (52 HS92),(52 HS92)
4,Other vegetable textile fibres (53 HS92),(53 HS92)
5,Man-made filaments (54 HS92),(54 HS92)
6,Man-made staple fibres (55 HS92),(55 HS92)
7,"Wadding, felt and nonwovens (56 HS92)",(56 HS92)
8,Carpets (57 HS92),(57 HS92)
9,Special woven fabrics (58 HS92),(58 HS92)
